In [3]:
import sys
sys.path.append('../')
import jax.numpy as jnp


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [4]:
import src.models.rnn as rnn
from src.utils.utils import *
import jax

In [5]:
trf=rnn.rnn_transform(rnn.MultiplicativeRNN,10,4,32)

In [6]:
key=jax.random.PRNGKey(0)
sample_o=jax.random.normal(key,[20,10])
sample_a=jax.random.normal(key,[20,4])
h_t=rnn.MultiplicativeRNN.initial_state(10,4,32,1)
rnn_params=trf.init(key,(sample_o[0],sample_a[0]),h_t)

In [7]:
for i in range(12):
    out,h_t=(trf.apply)(rnn_params,(sample_o[i],sample_a[i]),h_t)

In [75]:
def hvp(rnn_params,inputs,last_state,vector):
    def jvp(rnn_params,inputs,h_t,weights):
        jacobians=jax.jacrev(trf.apply)(rnn_params,inputs,h_t)[0]
        jvp=jax.tree_multimap(lambda x, y: jnp.tensordot(x,y,axes=y.ndim), jacobians, weights)
        jvp=jnp.stack(jax.tree_util.tree_flatten(jvp)[0],axis=0).sum(axis=0)
        return jvp
    hvp_fn=jax.jacrev(jvp)
    return hvp_fn(rnn_params,inputs,last_state,vector)

In [76]:
hvp=jax.jit(hvp)

In [77]:
import time

In [87]:
start=time.time()
h=hvp(rnn_params,(sample_o[0],sample_a[0]),h_t,rnn_params)
time.time()-start

0.019191741943359375

In [88]:
h

FlatMap({
  'multiplicative_rnn': FlatMap({
                          'b': DeviceArray([[[ 9.6794805e-03, -1.1301146e-02,  2.8433993e-02,
                                              -1.3579360e-02],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             ...,
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                           

### HVP: Pearlmutter's method

In [11]:
def hvp_pm(rnn_params,inputs,last_state,vector):
    def r_jacob(r,rnn_params,inputs,last_state,vector):
        theta_plus_rw=tree_sum(rnn_params,tree_scalar_multiply(vector,r))
        return jax.jacrev(trf.apply)(theta_plus_rw,inputs,last_state)[0]
    r=jnp.zeros(1)
    #return r_jacob(r,rnn_params,inputs,last_state,vector)
    return jax.jacfwd(r_jacob)(r,rnn_params,inputs,last_state,vector)

In [69]:
start=time.time()
h=hvp_pm(rnn_params,(sample_o[0],sample_a[0]),h_t,rnn_params)
time.time()-start

0.04781198501586914

In [28]:
h

FlatMap({
  'multiplicative_rnn': FlatMap({
                          'b': DeviceArray([[[[ 0.44770542],
                                              [-0.52271235],
                                              [ 1.3151586 ],
                                              [-0.6280867 ]],
                               
                                             [[ 0.        ],
                                              [ 0.        ],
                                              [ 0.        ],
                                              [ 0.        ]],
                               
                                             [[ 0.        ],
                                              [ 0.        ],
                                              [ 0.        ],
                                              [ 0.        ]],
                               
                                             ...,
                               
                                           